In [1]:
from __future__ import division
# 避免整数相除为0(上面这个前面不能有注释)
import operator
import numpy as np

# 从util引入获取训练集的函数
import sys
sys.path.append(r"F:\\FictionRecommend\\")
import Ipynb_importer
import process as read
import numpy as np
# 求矩阵的逆
from scipy.sparse.linalg import gmres

importing Jupyter notebook from process.ipynb


In [2]:
import test_read as mat_util

importing Jupyter notebook from test_read.ipynb
[[1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1]]


In [3]:
# 测试文件
log_file = "F:/badou/tmp/data/recommender/personalRank/log.txt"
# 评分文件
rating_file = "F:/badou/tmp/data/recommender/personalRank/ratings.csv"
# 用来看推荐效果
movies_file = "F:\\badou\\tmp\\data\\recommender\\data\\ml-latest-small\\movies.csv"

In [4]:
def personal_rank(graph, root, alpha, iter_num, recom_num = 10):
    """
    基础算法版本
    :param graph:
        根据用户的行为得到的用户和item的图结构
    :param root:
        将要给那个user推荐 root顶点
    :param alpha:
        以 alpha 的概率选择向下游走，以 1 - alpha 的概率选择回到起点
    :param iter_num:
        迭代次数
    :param recom_num:
        推荐个数(默认10个)
    :return:
        dict 长度为推荐个数 recom_num
            key -> item_id
            value -> pr值(personal rank)
    """
    # 存储所有的顶点对于root顶点的personal_rank值
    rank = {}
    # 初始值，除root顶点以外其余顶点personal_rank值都为0
    rank = {point: 0 for point in graph}
    # root顶点初始化为1
    rank[root] = 1
    # 输出的推荐dict
    recom_result = {}
    
    for iter_index in range(iter_num):
        # 用来存储当前迭代轮次下其余顶点对root顶点的pr值
        tmp_rank = {point: 0 for point in graph}
        
        # 上部分公式
        # 如果该顶点不是root顶点，则它的pr值 = 所有连接到该顶点的顶点将自己的pr值以 1/n 的概率贡献到该顶点上，
        # 那么n就是连接到该顶点的这个顶点的出度
        for out_point, out_dict in graph.items():
            for inner_point, value in graph[out_point].items():
                tmp_rank[inner_point] += round(alpha * rank[out_point] / len(out_dict), 4)
                # 下半部分公式：如果该顶点同时是root顶点的话，不仅有公式上半部分构成还要加上 1 - alpha
                if inner_point == root:
                    tmp_rank[inner_point] += round(1 - alpha, 4)
        
        # 完全相同时可以停止迭代(收敛)
        if tmp_rank == rank:
            break
        
        # 继续往下迭代
        rank = tmp_rank
    
    right_num = 0
    # 按pr值的得分进行排序，并过滤掉其中的user顶点和root顶点已经产生行为的item，从而得到推荐结果
    for zuhe in sorted(rank.items(), key = operator.itemgetter(1), reverse  = True):
        point, pr_score = zuhe[0], zuhe[1]
        # 如果顶点不是item顶点，需要过滤掉
        if len(point.split("_")) < 2:
            continue
        # 如果顶点是item顶点，但是和root顶点产生过行为，同样过滤掉
        if point in graph[root]:
            continue
            
        recom_result[point] = pr_score
        right_num  += 1
        if right_num > recom_num:
            break
    return recom_result

In [11]:
def get_one_user_recom():
    '''
    用户的推荐列表
    :return:
    '''
#     user = "A"
    user = "1"
    alpha = 0.6
    iter_num  = 100
    
    
    # {'item_c': 0.0726, 'item_e': 0.0363} 测试，明显 c > e
#     graph = read.get_graph_from_data(log_file)

    graph = read.get_graph_from_data(rating_file)
    recom_result = personal_rank(graph, user, alpha, iter_num)
    
    return recom_result

In [12]:
recom_result_base = get_one_user_recom()
print(recom_result_base)

{'item_318': 0.1281, 'item_296': 0.12249999999999991, 'item_589': 0.0881, 'item_858': 0.08149999999999996, 'item_2762': 0.06760000000000002, 'item_4993': 0.06500000000000002, 'item_150': 0.06460000000000002, 'item_32': 0.0631, 'item_588': 0.06010000000000002, 'item_364': 0.05910000000000001, 'item_7153': 0.05860000000000003}


In [13]:
# 矩阵算法版本
def personal_rank_maat(graph, root, alpha, recom_num = 10):
    '''
    :param graph:
        用户与item的二分图
    :param root:
        固定的顶点(为那个用户推荐)
    :param alpha:
        随机游走的概率
    :param recom_num:
        推荐的topn
    :return:
        dict
            key: item_id
            value: pr score
    '''
    # 得到M矩阵，所有顶点的集合以及每个顶点对应的行号
    m, vertex, address_dict = mat_util.graph_to_m(graph)
    # 需要推荐的用户顶点在顶点集合当中
    if root not in address_dict:
        return {}

    score_dict = {}
    recom_dict = {}

    # 单位阵E - alpha * m阵的转置.T
    mat_all = mat_util.mat_all_point(m, vertex, alpha)
    # 求mat_all => A矩阵的逆，及得到了所有顶点的推荐结果(线性代数)[求矩阵的逆即Ax = b]
    # 先得到root顶点的index即所对应的行号
    index = address_dict[root]
    # A * r = r0 (实际上就是一个n行一列的矩阵，只是将第index行初始化为 1，即root顶点对应所有顶点的pr值)
    initial_list = [[0] for row in range(len(vertex))]
    initial_list[index] = [1]
    r_zero = np.array(initial_list)
    # 解线性方程
    # 结果是一个元组
    # 第一维度是一个数组，标志着其余所有顶点对该root顶点的pr值
    res = gmres(mat_all, r_zero, tol = 1e-08)[0]
    for index in range(len(res)):
        # 此处的index 就是对应所有顶点的 index
        point = vertex[index]
        # 判断顶点是否是item顶点
        if len(point.strip().split("_")) < 2:
            continue
        # 排除已经产生行为过的顶点
        if point in graph[root]:
            continue
        # 推荐结果
        score_dict[point] = round(res[index], 3)
    # 按score排序去topn个
    for zuhe in sorted(score_dict.items(), key = operator.itemgetter(1), reverse = True)[:recom_num]:
        point, score = zuhe[0], zuhe[1]
        recom_dict[point] = score
    return recom_dict

In [14]:
# 测试矩阵算法的推荐结果
def get_one_user_by_mat():
    # user = "A"
    user = "1"
    alpha = 0.6
    # graph = read.get_graph_from_data(log_file)
    # recom_result = personal_rank_maat(graph, user, alpha)

    graph = read.get_graph_from_data(rating_file)
    recom_result = personal_rank_maat(graph, user, alpha, 100)
    return recom_result

In [15]:
recom_result_mat = get_one_user_by_mat()
recom_result_mat

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


{'item_1036': 0.001,
 'item_1046': 0.0,
 'item_106': 0.0,
 'item_106782': 0.0,
 'item_1077': 0.0,
 'item_1084': 0.0,
 'item_1086': 0.0,
 'item_1103': 0.0,
 'item_112552': 0.0,
 'item_1179': 0.0,
 'item_1193': 0.001,
 'item_1200': 0.001,
 'item_1221': 0.001,
 'item_122882': 0.0,
 'item_125': 0.0,
 'item_1258': 0.001,
 'item_131724': 0.0,
 'item_1387': 0.001,
 'item_150': 0.001,
 'item_1580': 0.001,
 'item_162': 0.0,
 'item_1704': 0.001,
 'item_176': 0.0,
 'item_1968': 0.001,
 'item_215': 0.0,
 'item_2288': 0.0,
 'item_232': 0.0,
 'item_26409': 0.0,
 'item_265': 0.0,
 'item_2762': 0.001,
 'item_2851': 0.0,
 'item_293': 0.001,
 'item_296': 0.002,
 'item_3024': 0.0,
 'item_318': 0.002,
 'item_319': 0.0,
 'item_32': 0.001,
 'item_34': 0.001,
 'item_342': 0.0,
 'item_345': 0.0,
 'item_348': 0.0,
 'item_364': 0.001,
 'item_368': 0.0,
 'item_377': 0.001,
 'item_380': 0.001,
 'item_4226': 0.001,
 'item_4306': 0.001,
 'item_4518': 0.0,
 'item_46970': 0.0,
 'item_475': 0.0,
 'item_48516': 0.0,
 '

In [16]:
# 看一下基础版本和矩阵版本推荐中有多少是相同的
num = 0
for ele in recom_result_base:
    if ele in recom_result_mat:
        num += 1
# 42
print(num)

11
